<a href="https://colab.research.google.com/github/Satvik-Gautam/Terrain-Recognition/blob/main/Random_forests_slip_and_rough.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mahotas
!pip install joblib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 15.0 MB/s eta 0:00:00


In [ ]:
!pip install pillow
!pip install --upgrade scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 30.8 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [ ]:
import numpy as np
from mahotas.features import haralick
import random
import cv2
import joblib
from PIL import Image
import os
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive



haralick_labels = ["Angular Second Moment",
                   "Contrast",
                   "Correlation",
                   "Sum of Squares: Variance",
                   "Inverse Difference Moment",
                   "Sum Average",
                   "Sum Variance",
                   "Sum Entropy",
                   "Entropy",
                   "Difference Variance",
                   "Difference Entropy",
                   "Information Measure of Correlation 1",
                   "Information Measure of Correlation 2",
                   "Maximal Correlation Coefficient"]


In [ ]:
s = ["grass" , "rocky" , "marshy" ,"sandy" ]
directory = [r"/content/drive/MyDrive/latest_data/", "/content/drive/MyDrive/terrain data/terrain data/"]
image_list = []
y_rough =[]
y_slip = []

for i in range(0,len(s)):
  for j in directory:

    folder = os.listdir(j + s[i])

    for file in folder:
          if file.endswith(('.png' ,'.PNG','.jpg' )):
            image_list.append(cv2.imread(os.path.join(j + s[i], file), cv2.IMREAD_GRAYSCALE))
          match i:
            case 0 :
              y_rough.append( 0 if random.uniform(0.2 ,0.3) < 0.5 else 1)
              y_slip.append( 0 if random.uniform(0.2,0.5)< 0.5 else 1)
            case 1 :
                y_rough.append( 0 if random.uniform(0.52,0.7)< 0.5 else 1)
                y_slip.append( 0 if random.uniform(0,0.2)< 0.5 else 1)
            case 2 :
                y_rough.append(0 if random.uniform(0.1,0.2)< 0.5 else 1)
                y_slip.append( 0 if random.uniform(0.8,1)< 0.5 else 1)
            case 3 :
                y_rough.append( 0 if random.uniform(0.3,0.48)< 0.5 else 1)
                y_slip.append( 0 if random.uniform(0.5,0.8)< 0.5 else 1)


In [ ]:
rows = len(image_list)
cols = 56

x = []

itr = 0
for i in image_list:

  h = haralick(i)
  arr = np.array(h).flatten()
  x.insert(itr  ,arr)

print(len(x))

2360


In [ ]:
#roughness
print(len(x))
print(len(y_rough))
# Split the dataset into training and testing sets
x_train_rough , x_test_rough , y_train_rough , y_test_rough = train_test_split(x , y_rough , test_size = 0.2 , random_state = 42)


# Create a Random Forest classifier
rf_classifier_rough = RandomForestClassifier(n_estimators=100, random_state=50)

# Train the classifier on the training data
rf_classifier_rough.fit(x_train_rough , y_train_rough)

# Make predictions on the test data
y_pred_rough = rf_classifier_rough.predict(x_test_rough)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test_rough, y_pred_rough)
print(f"Accuracy: {accuracy:.2f}")
joblib.dump(rf_classifier_rough, 'random_forest_rough_latest.joblib')

2360
2360
Accuracy: 0.80


['random_forest_rough_latest.joblib']

In [ ]:
#slipperiness
x_train_slip , x_test_slip , y_train_slip , y_test_slip = train_test_split(x , y_slip , test_size = 0.2 , random_state = 42)


# Create a Random Forest classifier
rf_classifier_slip = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier on the training data
rf_classifier_slip.fit(x_train_slip , y_train_slip)

# Make predictions on the test data
y_pred_slip = rf_classifier_slip.predict(x_test_slip)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test_slip, y_pred_slip)
print(f"Accuracy: {accuracy:.2f}")
joblib.dump(rf_classifier_slip, 'random_forest_slip_latest.joblib')

Accuracy: 0.88


['random_forest_slip_latest.joblib']

In [ ]:
image = cv2.imread(r"/content/drive/MyDrive/terrain data/terrain data/grass/grass (1)_rotate.jpg", cv2.IMREAD_GRAYSCALE)
har = np.array(haralick(image)).flatten()
a = []
a.append(har)
predictions_rough = rf_classifier_rough.predict(a)
predictions_slip = rf_classifier_slip.predict(a)
print("roughness"  + str(predictions_rough))
print("slipperiness" + str(predictions_slip))

roughness[0]
slipperiness[1]


roughness - 0 = not rough , 1 = rough
slipperiness - 0 = not slippery , 1 = slippery